# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [3]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [5]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [6]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-CoL2wRsHTil1XkLTQWkXExRKkkB56',
 'object': 'chat.completion',
 'created': 1766113338,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: On Venus, a day is longer than its year. It takes about 243 Earth days to rotate once, but only about 225 Earth days to orbit the Sun.',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 494,
  'total_tokens': 505,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 448,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [4]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: Bananas are technically berries, while strawberries aren’t. Botanically, a berry comes from a single ovary, and bananas fit that bill, whereas strawberries are aggregate fruits formed from many ovaries of a single flower. Want another fun fact?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [7]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: A day on Venus is longer than a year on Venus. Venus spins very slowly and in the opposite direction, taking about 243 Earth days to rotate once, but only about 225 Earth days to orbit the Sun.'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [11]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [12]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Did you know that a group of pugs is called a "grumble"? How adorable is that!'

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [13]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [14]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 3.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 7.9 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  13 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  20 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  25 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  30 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   2% ▕                  ▏  37 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   2% ▕                  ▏  42 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   2% ▕       

In [15]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [17]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Here's one:\n\nDid you know that honey never spoils? Archaeologists have found pots of honey in ancient Egyptian tombs that are over 3,000 years old and still perfectly edible! Honey's unique composition, with its low moisture content and high acidity, makes it nearly impossible for bacteria and other microorganisms to grow, which is why it can last virtually indefinitely. Isn't that sweet?"

In [18]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 281 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 1.7 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 7.4 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  13 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  16 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  21 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  29 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  33 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   4% ▕                  ▏  39 MB/1.1 GB            

In [19]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'Here are three engaging and thoughtfully fun facts:\n\n1. **The Matrix**: Named by mathematicians in tech terms due to its underlying technological structure, referencing the "matrix" concept in geometry.\n\n2. **Inception**: Referenced by Frank Miller in his book series *Interstellar*, which also has a movie adaptation with the same title (though it\'s a different story).\n\n3. **Coco**: Based on the song from *The Beatles*, renamed due to its association with Coca-Cola, "converging flow."\n\nEach of these ties back to innovative concepts and cultural references!'

In [20]:
from IPython.display import Markdown, display

In [22]:
# Step 1: Create your prompts

system_prompt = """
Your job is to concisely summarize the text provided
Please ignore text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt = """
Conversation opened. 1 unread message.

Skip to content
Using Gmail with screen readers
1 of 214
Holocaust and Heresy: When History Becomes Holy
Inbox

JD Hall from Insight to Incite <insighttoincite@substack.com> Unsubscribe
4:10 PM (6 minutes ago)
to me

Forwarded this email? Subscribe here for more

 
Insight to Incite: Open Source Intelligence Analysis 		
Insight to Incite: Audio Vers…
Holocaust and Heresy: When Hi…
0:00		
11:16				
 
Listen now
 
Holocaust and Heresy: When History Becomes Holy
The Webbon Controversy and the Religion Behind the Post-War Consensus
JD Hall
Dec 18

 




READ IN APP
 

The backlash against pastor Joel Webbon for suggesting that Holocaust teaching now functions like a religion revealed more than his critics intended. The speed of the outrage and the calls for excommunication exposed a system that does not answer questions with argument, but with punishment. That is not how history operates. It is how creeds operate.

This article argues that the Holocaust has become the moral cornerstone of the Post-War Consensus, a civil religion that has shaped the West since 1945. Within that system, the Holocaust functions as original sin, the Allied victory as redemption, and post-war institutions as guardians of salvation. Questioning that framework is treated as heresy because it threatens the authority of the entire order.

Christians, trained to recognize counterfeit gospels, should have noticed this immediately. Many did not. The Webbon episode exposes how deeply the Post-War Consensus has catechized the church, and why discernment is now more necessary than ever.

Every civilization has moments it remembers, and every civilization eventually chooses which moments it is no longer allowed to examine. The transition is subtle. At first, the event is taught as history. Later, it is taught as moral instruction. Finally, it is taught as identity itself. Once that transition is complete, the event no longer functions as history at all. It becomes religious infrastructure.

A days weeks ago, pastor Joel Webbon was attacked by a Temu version of Media Matters for making an observation that should not have been controversial in a free society and certainly should not have been incomprehensible to Christians. He suggested that Holocaust teaching in the modern West has taken on the characteristics of a religion. He did not deny the event, minimize suffering, or dispute certain details He observed that the way the Holocaust is taught, protected, ritualized, and enforced now resembles a sacred system rather than a historical one.

He didn’t stumble into a historical dispute, when he uttered the lines that RightWingWatch would use to cut from context and disseminate to the outrage mob. He brushed against a sacred boundary. The reaction was not disagreement, correction, or counterargument. It was denunciation, moral alarm, and calls for ecclesiastical expulsion and excommunication. And that makes sense, because when history becomes religious creed, that produces outrage.

One of the clearest markers of religious catechism is rigidity. In ordinary scholarship, approximation is normal. Estimates vary, language shifts, emphasis differs. In catechism, precision is not about clarity but loyalty. The answer must be recited correctly and precisely, not because other answers are false, but because deviation signals rebellion against the religious order.

Christians understand this instinctively. We were raised on questions that demand fixed responses. What is justification? What is the chief end of man? What is the purpose of God’s law? The point is not that paraphrase is impossible, but that catechesis is designed to bind a community together through shared language. The words matter because the words signal membership.

And that’s why when asked how many Jews died in the Holocaust you aren’t permitted to say a bunch, or millions, or almost six million. You must say six million. Anything less is heresy. It’s holocaust denialism, and that’s a venial sin. And that’s because it’s become a religion. But that’s just one religion undertone that’s developed among many.


IF YOU PREFER TO LISTEN ON SPOTIFY, CLICK HERE
ORIGINAL SIN WITHOUT ATONEMENT

Every religion has a doctrine of guilt. The only question is whether it has a doctrine of redemption.

In Christianity, guilt is named clearly and traced honestly. It is universal, inherited, and inescapable. But it is also resolved somehow, on the cross. The entire system turns on the claim that guilt does not persist forever if atonement is offered for it. It is absorbed, answered, and propitiated, with God’s wrath ultimately extinguished. The cross is not an ongoing accusation but a finished verdict.

Civil religion does the opposite. It names guilt, then preserves it because guilt is very important to accomplishing the desired political goals. It teaches moral inheritance without moral resolution. The past becomes a permanent stain on the present. The only reason reparations haven’t gained more favor is because it would signal that some debts could be paid, which defeats the purpose of the entire exercise. Responsibility and blame must go on forever. Penance is perpetual.

This is where modern Holocaust pedagogy quietly departs from historical instruction and enters theological space. The event is not simply remembered, but it is transmuted into a standing moral debt that must be serviced indefinitely, and imputed to anyone who doesn’t follow the religious rules set forth. Political loyalty, foreign policy alignment, financial support, and rhetorical obedience become ways of proving that the debt is still being honored.

This is not how history operates. But, it is how sacralized guilt operates.

Share

Christians should notice the structural inversion immediately. A system that teaches inherited moral obligation without closure is not merely educational. It is eschatological. It has an end times vision, but no final judgment. The work of holocaust memorialism is never done. It’s useful in literally every single political discussion. The threat is never past. It must be brought up endlessly and in every conversation, and settle every dispute. It’s doctrine, after all.

The slogan “never again” functions less like a lesson and more like a creed. It does not point forward to restoration. It points sideways to perpetual management. Salvation is no longer something accomplished. It is something administered gradually, so long as people go along with whatever political action we are told best memorializes the Holocaust and best prevents another. Truth or positional correctness takes back-seat to stopping the next Holocaust that we are told is just around the corner if people express inappropriate opinions or have un-authorized thoughts.


WHY “WHY” IS FORBIDDEN

Another unmistakable marker of religious sacralization is the prohibition on motive analysis.

In every other historical atrocity, the first task of the historian is explanation. Causes are examined. Conditions are weighed. Ideologies are traced. Institutional failures are identified. None of this is understood as sympathy. It is understood as the responsibility of historians. If you do not understand why something happened, you cannot prevent it from happening again, obviously. But that’s not how it works with the Holocaust at all. Instead, historians are tasked as the guardians of society, making sure no one goes back down the coridoors of time without a permit - almost never issued - for fear of what might be found there. No, the job of historians in regard to the Holocaust is to gate-keep any such historical explanations, or else another Holocaust is right around the corner. The canon of history was shut about a decade after the war ended (by the victors), and any attempt to open it again is treated with contempt. It’s similar to the way that the Roman Catholic Church has made historic claims but doesn’t exactly break out the relics for investigation. Just trust them, because that’s really a vial of Mary’s breast milk and that’s all there is to it.

With religious events, explanation itself becomes suspect. Asking why is treated as an act of aggression against the faith. Structural analysis is framed as excuse. Context is treated as contamination. Evil must remain abstract or the entire moral architecture that was erected immediately following WWII begins to wobble.

This is not how Scripture handles evil. Scripture names wickedness without hesitation, but it never refuses to analyze it. Pharaoh is not simply called evil. His heart is hardened through pride, fear, and power. Babylon is not merely condemned. Its economic, religious, and political structures are exposed. Rome is not reduced to a monster. It is dissected as an empire that worships force and order. In the last 20 years, no shortage of essays, articles, and books have hypothesized why Judas betrayed Christ. Was it greed, or was he trying to force Christ into a Maccabee-style insurrection to become a political Messiah? You get the point.

Christian theology does not fear explanation because it does not fear truth. False systems fear explanation because they’re built on lies or twisted truths.


Join JD Hall’s subscriber chat
Available in the Substack app and on web
Join chat
When a society forbids “why,” it is not protecting victims. It is protecting a narrative. It is protecting the system that has grown up around the narrative. That system depends on moral clarity without intellectual clarity, and it cannot survive sustained inquiry.

This is why the reaction to Webbon mattered. He did not deny an event. He noticed a pattern that suggests a religion was forming. The response was not to correct his claim but to punish his observation and declare him a heretic (as aspiring evangelical influencers, Joe Boot and James White, literally charged him) with obvious religious zeal. That response tells you that the Webbons’s observation obviously landed.

WHEN THE PRIESTHOOD DEMANDS EXCOMMUNICATION

Every religion eventually reveals itself by how it enforces boundaries.

The most revealing moment in the Webbon controversy was not the criticism itself. It was the demand for ecclesiastical discipline, shunning, and excommunication. Secular activists and ideological watchdogs merely denounced him, sort of a case-in-point, “Here’s what those crazy conservatives are saying today” type thing. But the self-appointed Sanhedrin of Concerned Evangelicals™ called for his removal from ministry. They demanded excommunication.

These are not church members appealing to elders. These are not theologians correcting doctrine. These are outsiders invoking religious penalties to enforce compliance with a civil taboo. They deny being a religion while demanding the church act as their enforcement arm. It’s pretty crazy to watch.

The self-appointed gurus of Internet Apologetics saw the RightWingWatch clip and instead of evaluating Webbon’s claims, they instinctively thought, “Oh, good. We can attack our enemy with this.” And they did so while they should have acknowledged on the credibility of limited expertise that “Yes, in fact, the Holocaust is taking on religious undertones.”

Leave a comment

Christians have seen this before. Empires have always tried to harness the church’s disciplinary mechanisms for their own purposes. The novelty here is that the demand comes from people who insist no religion is involved. They want the punishment without the confession, the banishment without the creed. The irony is thick enough to butter toast with.

What makes this episode especially embarrassing for the religious class is that they played along. Pastors and commentators rushed to distance themselves, to signal compliance, to assure the watching world that they too recognized the violation of decorum. Very few stopped to ask why a sociological observation had triggered a call for religious banishment.

That failure is not primarily intellectual. It is spiritual. Christians are trained to recognize false gospels, counterfeit righteousness, and borrowed moral authority. They are supposed to be alert to systems that demand loyalty while denying grace. Instead, many joined the chorus, mistaking fear for faithfulness.

If the Holocaust were functioning only as history, the response would have been historical. Evidence would have been marshaled. Pedagogical intent would have been explained. Claims would have been weighed. Instead, the response took the form reserved for blasphemy. Webbon was not treated as mistaken. He was treated as dangerous.

This is what religious systems do when touched at the nerve. They do not debate. They guard. They do not persuade. They enforce. The people who reacted most strongly were not defending facts, but policing a boundary that must not be crossed if the moral order is to remain intact.

What makes the episode almost comical is that the reaction did more to validate Webbon’s claim than any argument he could have offered. If you want to convince people that something has become religious, demand excommunication when someone notices it. Nothing proves the point faster.

The irony should not be missed. These are men who will spend hours policing doctrinal language, warning against imprecision, and dissecting sermons line by line. They will warn young men about therapeutic moralism, cultural Marxism (minus explanation about who started it), and progressive Christianity. But when a civil religion demands obedience, they fold.

Why? Because discernment is costly when it cuts against the grain of the age. It is easier to critique fringe movements than to question the moral architecture that props up respectable power. It is easier to spot false teaching in a church plant than to recognize catechesis coming from museums, media, and public education.

The religious class has grown accustomed to guarding the edges while ignoring the center. They know how to fight small errors but have lost the nerve to question foundational assumptions. As a result, they enforce boundaries they did not establish and defend taboos they did not examine.

Message JD Hall

ZEAL WITHOUT KNOWLEDGE, DISCIPLINE WITHOUT SHEPHERDING

One of the most revealing aspects of the reaction to Webbon was how quickly the language of discipline appeared. Calls for removal, condemnation, and separation came easily.

Church discipline is a serious thing. It exists for the protection of doctrine and the restoration of sinners. It is meant to be exercised by elders who know their flock and understand the difference between error and rebellion. When discipline is invoked as a signaling mechanism to satisfy outside outrage, it ceases to be discipline at all. It becomes theater. What happened here was not shepherding, but cultural appeasement.

The demand was not that Webbon repent of sin. It was that he retract an observation. Sin calls for repentance. Observation calls for discussion. When observation is treated as sin, the system has already decided that certain thoughts are forbidden regardless of intent or accuracy. And this is exactly how civil religions maintain order.

Christians who participate in this process are not being discerning. They are being useful.

KNOWING THE SIGNS OF THE TIMES

Jesus rebuked the religious leaders of His day for being able to read the weather but not the moment they were living in. He accused them of blindness, not ignorance. They knew Scripture. They knew tradition. They simply failed to recognize how power was operating in front of them.

That warning applies here.

We are living in a moment when secular systems increasingly demand religious loyalty while denying religious status. They insist on moral conformity, enforce linguistic precision, punish heresy, and demand sacrifice, all while claiming neutrality. Christians who cannot recognize that pattern are not cautious. They are asleep.

The Webbon episode was a test, and many failed it. Not because they denied Christ, but because they could not tell the difference between the Gospel and a rival creed. They mistook moral panic for righteousness and enforcement for truth.

This is not a call to deny history. It is a call to refuse idolatry. Christians are commanded to remember truthfully, to honor the dead, and to pursue justice. They are not commanded to uphold a civil religion that replaces grace with memory and redemption with management.

Christians should be grateful to live in a time when these things are visible. The exposure is a mercy. False gods always demand silence. They always demand sacrifice. They always reveal themselves eventually.


THE BIGGER PICTURE

The reason the reaction to Joel Webbon was so swift and so hysterical is not because the Holocaust stands alone. It is because it sits at the center of a much larger religious system that has governed the West since 1945. The Holocaust is not merely remembered within that system. It functions as the original sin that explains everything that followed.

What Christians often miss is that the Post-War Consensus is not simply an agreement about foreign policy or international cooperation. It is a moral cosmology. It tells a story about the fall of man, the redemption of civilization, and the institutions that must be obeyed if the world is to remain saved. The Holocaust is the catastrophe that gives the story its authority. The Allied victory is the redemptive act. Everything after that is downstream theology.

This is why certain institutions are treated as sacred. NATO is not defended on the basis of prudence or success. It is defended as a guardian of peace. The United Nations is not evaluated by its fruit. It is assumed to be indispensable. Global finance is not weighed against local harm. It is justified as stabilizing the world. These things are not debated as policies. They are defended as pillars holding up the moral order born out of 1945.

Once that framework is understood, the Webbon episode makes perfect sense. Questioning the religious function of Holocaust pedagogy does not merely challenge an educational approach. It threatens the entire moral narrative that justifies the post-war world. If the Holocaust can be discussed as history rather than sacred trauma, then the authority it grants to the post-war order weakens. If that authority weakens, then the institutions built on it must answer for themselves.

That is why the system cannot tolerate analysis. It does not fear denial. It fears desacralization.

THE SAINTS, THE REDEEMERS, AND THE UNTOUCHABLES

Every religion has its saints, and the Post-War Consensus is no exception. The men who presided over the Allied victory are not treated as statesmen who made hard choices under pressure. They are treated as redeemers of civilization. Churchill, Roosevelt, Truman, and Eisenhower are insulated from moral scrutiny in ways no other political figures enjoy.

Their decisions are not weighed in proportion. Their failures are reframed as necessities. Their atrocities are explained away as tragic but unavoidable. To question them is to violate the creed. To suggest that they acted with cruelty or recklessness is to commit heresy.

This is why conservative Christians, who would never tolerate hagiography in church history, suddenly bristle when war leaders are examined honestly. They can dissect Augustine, Calvin, or Luther without flinching, but become defensive the moment someone questions Churchill or Nagasaki. They have learned, often without realizing it, which saints may not be touched.

The Holocaust plays a crucial role here. It functions as the justification for absolution. Because the enemy was absolute evil, the redeemers must be treated as absolute good. Once that logic is accepted, moral inquiry becomes dangerous. If the redeemers are sinners like everyone else, then the redemption narrative collapses.

This is not history. It is theology.

WHY CONSERVATIVE CHRISTIANS DEFEND WHAT THEY SHOULD QUESTION

Perhaps the most unsettling aspect of the Post-War Consensus is how deeply it has catechized conservative Christians. These are people who pride themselves on skepticism toward media, government, and academia. They know propaganda exists. They know narratives are constructed. Yet when it comes to the post-war story, they suddenly insist that everything is settled.

They will defend Harry Truman with more passion than they defend the sanctity of life. They will bristle at criticism of Nagasaki more than they bristle at open mockery of Christ. They will accuse fellow Christians of moral depravity for asking questions they would ask about any other war.

The reason is simple. They have mistaken the Consensus for reality itself. It is not something they believe. It is something they inhabit. It is the air they breathe. Challenging it feels like destabilizing the world, not revising an argument.

This is why the Holocaust occupies such a powerful place within the system. It anchors the entire moral universe. If that anchor is loosened, everything else must be justified on its own merits. Institutions must answer for their failures. Wars must be weighed honestly. Power must be restrained rather than sanctified.

That prospect terrifies people who have confused moral order with moral narrative.

WHY THIS MATTERS NOW

The Post-War Consensus is aging. Its institutions are failing. Its promises ring hollow. Yet its moral authority remains fiercely guarded, because without that authority the system has nothing left.

Christians should be the first to recognize this moment. They know what false gospels look like. They know what happens when history is turned into creed and power is baptized as righteousness. They know that any competing system of religion demanding loyalty without question is ultimately hostile to the Gospel.

The tragedy is not that secular institutions behave religiously. The tragedy is that the church often cannot tell the difference anymore.

That is why controversies like this matter. They force the issue into the open. They reveal who is enforcing which creed, and at what cost. They remind us that remembering truthfully is not the same thing as worshiping history, and that honoring victims does not require surrendering discernment.

Christ alone is King. Every other order, no matter how loudly it invokes moral necessity, must answer to that truth.

If you like my free content, you’ll love my exclusive content. Please grab a 20% discount to subscribe and get the full archives (and so much more).

Get 20% Off

If you don’t do subscriptions, please consider a one-time gift by clicking the ‘coffee link’ below


.

Invite your friends and earn rewards
If you enjoy Insight to Incite: Open Source Intelligence Analysis , share it with your friends and earn rewards when they subscribe.
Invite Friends

 
Like
Comment
Restack
 
...

[Message clipped]  View entire message

"""

# Step 2: Make the messages list


messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

# Step 3: Call OpenAI
# response =
# response = openai.chat.completions.create(model="deepseek-r1:1.5b", messages=messages)
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=messages)
response.choices[0].message.content

# Step 4: print the result
# print(
display(Markdown(response.choices[0].message.content))

TheConversation between Joel Joel Webbon and others delves into how the Holocaust has become a key narrative element in the evolution of Christian views. Here's a structured summary:

1. **Webbon's Perspective**: Joel Webbon is presenting the Holocaust as a key lesson from resistance against colonizing powers, offering a moral framework for how history should be understood.

2. **Modern Religious Interpretations**:
   - Most Christians today examine the Holocaust using historical narratives, media representations, and authority figures rather than just faith-based approaches.
   - These perspectives reflect growing internal scrutiny as media and academia change the interpretation of events.

3. **Challenges in Modern Settings**: The assistant highlights how authorities often prioritize their positions over historical accuracy, questioning controversial topics like the Holocaust's causes or justifications.

4. **The Post-War Consensus System**:
   - Post-War Consensus institutions grow outdated, with its focus and claims becoming irrelevant.
   - Their ideals are challenged by external pressures, prompting examination of historical narratives as if saving grace lacks foundation.

5. **Historical Memory and Critical examination**: The Holocaust's pivotal role as a memory is preserved despite changes in teaching methods. This ties into how media shapes public opinion without fully conveying facts, aligning with the abstract nature of addressing external pressures versus internal questioning.

6. **Abstractness vs. Embraceability**: As abstract forms, these perspectives highlight how we engage with history through faith and authority, rather than historical fact or modern institutions. They are part of a broader spiritual tapestry, influencing legal debates internally rather than externally.

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`